In [3]:
import sys
sys.path.append('/home/mdrnevich/carl-torch')

import json
import os
import uproot

import htcondor
import classad

In [4]:
with open("carl_data_metadata.json", 'r') as f:
    data_metadata_dict = json.load(f)
    max_data_index = data_metadata_dict["max_data_index"]
    max_jet_size = data_metadata_dict["max_jet_size"]
    max_electron_size = data_metadata_dict["max_electron_size"]
    max_muon_size = data_metadata_dict["max_muon_size"]

In [5]:
jet_features = ["Jet_pt", "Jet_eta", "Jet_phi", "Jet_mass"]
electron_features = ["Electron_pt", "Electron_eta", "Electron_phi", "Electron_mass"]
muon_features = ["Muon_pt", "Muon_eta", "Muon_phi", "Muon_mass"]

In [6]:
features = [jet_features[0] + "{}".format(i) for i in range(max_jet_size)]
features.extend([jet_features[1] + "{}".format(i) for i in range(max_jet_size)])
features.extend([jet_features[2] + "{}".format(i) for i in range(max_jet_size)])
features.extend([jet_features[3] + "{}".format(i) for i in range(max_jet_size)])

features.extend([electron_features[0] + "{}".format(i) for i in range(max_electron_size)])
features.extend([electron_features[1] + "{}".format(i) for i in range(max_electron_size)])
features.extend([electron_features[2] + "{}".format(i) for i in range(max_electron_size)])
features.extend([electron_features[3] + "{}".format(i) for i in range(max_electron_size)])

#features.extend([muon_features[0] + "{}".format(i) for i in range(max_muon_size)])
#features.extend([muon_features[1] + "{}".format(i) for i in range(max_muon_size)])
#features.extend([muon_features[2] + "{}".format(i) for i in range(max_muon_size)])
#features.extend([muon_features[3] + "{}".format(i) for i in range(max_muon_size)])

In [7]:
#features = jet_features + electron_features #+ muon_features

In [8]:
features

['Jet_pt0',
 'Jet_pt1',
 'Jet_pt2',
 'Jet_pt3',
 'Jet_pt4',
 'Jet_pt5',
 'Jet_pt6',
 'Jet_pt7',
 'Jet_pt8',
 'Jet_pt9',
 'Jet_pt10',
 'Jet_pt11',
 'Jet_pt12',
 'Jet_pt13',
 'Jet_pt14',
 'Jet_pt15',
 'Jet_pt16',
 'Jet_pt17',
 'Jet_pt18',
 'Jet_pt19',
 'Jet_pt20',
 'Jet_pt21',
 'Jet_pt22',
 'Jet_pt23',
 'Jet_pt24',
 'Jet_pt25',
 'Jet_pt26',
 'Jet_pt27',
 'Jet_pt28',
 'Jet_pt29',
 'Jet_pt30',
 'Jet_pt31',
 'Jet_pt32',
 'Jet_pt33',
 'Jet_pt34',
 'Jet_pt35',
 'Jet_pt36',
 'Jet_pt37',
 'Jet_pt38',
 'Jet_pt39',
 'Jet_pt40',
 'Jet_pt41',
 'Jet_pt42',
 'Jet_pt43',
 'Jet_pt44',
 'Jet_pt45',
 'Jet_pt46',
 'Jet_pt47',
 'Jet_pt48',
 'Jet_pt49',
 'Jet_pt50',
 'Jet_pt51',
 'Jet_pt52',
 'Jet_pt53',
 'Jet_pt54',
 'Jet_pt55',
 'Jet_pt56',
 'Jet_pt57',
 'Jet_pt58',
 'Jet_pt59',
 'Jet_pt60',
 'Jet_pt61',
 'Jet_pt62',
 'Jet_pt63',
 'Jet_eta0',
 'Jet_eta1',
 'Jet_eta2',
 'Jet_eta3',
 'Jet_eta4',
 'Jet_eta5',
 'Jet_eta6',
 'Jet_eta7',
 'Jet_eta8',
 'Jet_eta9',
 'Jet_eta10',
 'Jet_eta11',
 'Jet_eta12',
 'Jet_

In [9]:
#weight_features = ["weight_mc", "weight_pileup", "weight_leptonSF", "weight_jvt", "weight_bTagSF_DL1r_Continuous"]
weight_features = ["genWeight", "btagWeight_CSVV2"]

In [10]:
#os.system("bash ~/.bash_profile; bash setup_training.sh; python ~/carl-torch/train.py -g ttbar_PS_var_classifier -n CMS_ttbar_nominal_training_data -v CMS_ttbar_PS_var_training_data -p /data/mdrnevich/AGC/ -e -1 -f {} -t nominal_Loose --weightFeature weight_mc_combined --nepoch 10 -l 200 200 200 200 200".format(','.join(features)))

In [11]:
n_events = int(1e6)
n_layers = 3
n_nodes = 1000
n_epochs = 15
model_name = "ttbar_PS_var_classifier_{}events_{}layers_{}nodes_{}epochs".format(n_events, n_layers, n_nodes, n_epochs)

with open("carl_training_script.sh", 'w') as  f:
    print("#!/usr/bin/bash", file=f)
    print("source /home/mdrnevich/.bashrc", file=f)
    print("source /home/mdrnevich/.bash_profile", file=f)
    print("conda activate carl-for-agc", file=f)
    print("cd /data/mdrnevich/AGC/training", file=f)
    print("python /home/mdrnevich/carl-torch/train.py -g {} -n CMS_ttbar_nominal_training_data_CARL -v CMS_ttbar_PS_var_training_data_CARL -p /data/mdrnevich/AGC/ -e {} -f {} -t Events --weightFeature weight_mc_combined --nepoch {} -l {}".format(
            model_name,
            n_events,
            ','.join(features),
            n_epochs,
            ' '.join([str(n_nodes)]*n_layers)),
        file=f)
    print("python /home/mdrnevich/carl-torch/evaluate.py -g {} -n CMS_ttbar_nominal_training_data_CARL -v CMS_ttbar_PS_var_training_data_CARL -p /data/mdrnevich/AGC/ -e {} -f {} -t Events --weightFeature weight_mc_combined".format(
            model_name,
            n_events,
            ','.join(features)),
        file=f)

In [38]:
hostname_job = htcondor.Submit({
    "executable": "carl_training_script.sh",
    "output": "jobs/carl_training_job.$(Cluster).$(Process).out",
    "error": "jobs/carl_training_job.$(Cluster).$(Process).err",
    "log": "jobs/carl_training_job.log",
    "request_cpus": "1",
    "request_memory": "64GB"
})

In [39]:
schedd = htcondor.Schedd()                   # get the Python representation of the scheduler
submit_result = schedd.submit(hostname_job)  # submit the job
print(submit_result.cluster())               # print the job's ClusterId

HTCondorLocateError: Unable to locate local daemon

In [31]:
n_events = int(1e6)
n_layers = 3
n_nodes = 500
n_epochs = 15
model_name = "ttbar_PS_var_classifier_{}events_{}layers_{}nodes_{}epochs".format(n_events, n_layers, n_nodes, n_epochs)

with open("carl_evaluate_script.sh", 'w') as  f:
    print("#!/usr/bin/bash", file=f)
    print("source /home/mdrnevich/.bashrc", file=f)
    print("source /home/mdrnevich/.bash_profile", file=f)
    print("conda activate carl-for-agc", file=f)
    print("cd /data/mdrnevich/AGC/training", file=f)
    print("python /home/mdrnevich/carl-torch/evaluate.py -g {} -n CMS_ttbar_nominal_training_data_CARL -v CMS_ttbar_PS_var_training_data_CARL -p /data/mdrnevich/AGC/ -e {} -f {} -t Events --weightFeature weight_mc_combined".format(model_name, n_events, ','.join(features)), file=f)

In [24]:
' '.join([str(n_nodes)]*n_layers)

'500 500 500'